## Model for style transfer accuracy

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Машинка/lemmatized.csv')

In [5]:
data = data[['comment', 'toxic']]
data['id'] = np.arange(len(data))
data.head()

,comment,toxic,id
0,дворник уничтожать,1,0
1,старший неделя шипеть принимать подкидыш котор...,0,1
2,полностью согласный,0,2
3,нога вверх ничто изменяться,0,3
4,значить левый ребенок,0,4


In [6]:
data.isnull().sum()

comment    665
toxic        0
id           0
dtype: int64

In [7]:
data = data.dropna()

In [8]:
# we cannot train on full data, so we'll just take 30K toxic + 30K non-toxic from it (out of 160K)
import copy
data_full = copy.deepcopy(data)
data_toxic = data_full.loc[data_full['toxic'] == 1]
data_nontoxic = data_full.loc[data_full['toxic'] == 0].sample(n = len(data_toxic))
data = pd.concat([data_toxic, data_nontoxic], ignore_index = True)
data.shape

(62806, 3)

In [9]:
texts = list(data['comment'])
labels = list(data['toxic'])

In [10]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.3, random_state=42)

In [11]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=32,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }

In [46]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

In [47]:
train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
val_dataset = CustomDataset(val_texts, val_labels, tokenizer)

In [48]:
batch_size = 16
num_epochs = 10

In [49]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [50]:
model = BertForSequenceClassification.from_pretrained("DeepPavlov/rubert-base-cased-conversational", num_labels=2)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassi

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: ignored

In [52]:
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [53]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for i, batch in enumerate(train_dataloader):
        if batch is None:
          print(i, 'huinia')
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)

    # Validation loop
    model.eval()
    total_val_loss = 0
    total_val_correct = 0

    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            total_val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            total_val_correct += torch.sum(predictions == labels).item()

    avg_val_loss = total_val_loss / len(val_dataloader)
    val_accuracy = total_val_correct / len(val_dataset)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print("")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/Машинка/model")
tokenizer.save_pretrained("/content/drive/MyDrive/Машинка/tokenizer")

Epoch 1/10
Train Loss: 0.1873
Validation Loss: 0.1618
Validation Accuracy: 0.9406

Epoch 2/10
Train Loss: 0.1065
Validation Loss: 0.1767
Validation Accuracy: 0.9377

Epoch 3/10
Train Loss: 0.0597
Validation Loss: 0.2154
Validation Accuracy: 0.9360

Epoch 4/10
Train Loss: 0.0346
Validation Loss: 0.2333
Validation Accuracy: 0.9274

Epoch 5/10
Train Loss: 0.0256
Validation Loss: 0.3359
Validation Accuracy: 0.9358

Epoch 6/10
Train Loss: 0.0189
Validation Loss: 0.3428
Validation Accuracy: 0.9346

Epoch 7/10
Train Loss: 0.0156
Validation Loss: 0.3335
Validation Accuracy: 0.9338

Epoch 8/10
Train Loss: 0.0146
Validation Loss: 0.4096
Validation Accuracy: 0.9342

Epoch 9/10
Train Loss: 0.0176
Validation Loss: 0.3497
Validation Accuracy: 0.9323

Epoch 10/10
Train Loss: 0.0142
Validation Loss: 0.3216
Validation Accuracy: 0.9319



('/content/drive/MyDrive/Машинка/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Машинка/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Машинка/tokenizer/vocab.txt',
 '/content/drive/MyDrive/Машинка/tokenizer/added_tokens.json')

## Style transfer accuracy

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
def STA(tokenizer, model, preds, true_labels):
  """
  Input:

  tokenizer: BERT tokenizer

  model: BERT model

  preds: list of strings

  --------------------------

  Return:

  accuracy, embeddings

  """

  inputs = tokenizer.batch_encode_plus(
    preds,
    add_special_tokens=True,
    max_length=32,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
    )

  input_ids = inputs["input_ids"].to(device)
  attention_mask = inputs["attention_mask"].to(device)

  model.eval()
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)

  logits = outputs.logits
  predictions = torch.argmax(logits, dim=1)

  # store embeddings for cosine similarity later
  embeddings = outputs[0]

  true_labels_tensor = torch.from_numpy(true_labels).long().to(device)


  accuracy = accuracy_score(true_labels_tensor.cpu(), predictions.cpu())

  return accuracy, embeddings

Let's calculate STA for our predictions:

## BLEU

> The metric will show how close we stored the content

In [32]:
from nltk.translate.bleu_score import sentence_bleu

In [33]:
def tokenize(texts):
  result = []
  for text in texts:
    tokenized = text.split()
    result.append(tokenized)
  return result

In [34]:
def bleu_score(true_texts, pred_texts):
  true_list = tokenize(true_texts)
  pred_list = tokenize(pred_texts)

  bleu_scores = []
  for i in range(len(true_list)):
    reference = [true_list[i]]
    candidate = pred_list[i]
    bleu = sentence_bleu(reference, candidate)
    bleu_scores.append(bleu)

  avg_bleu = np.mean(bleu_scores)

  return bleu_scores, avg_bleu

Now we calculate BLEU scores:

## Cosine similarity

Again for checking if we stored content:

We already stored embeddings for predicted texts from BERT. Now we need to calculate them for our original data (we'll also take sample not full data)

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
def embeddings(texts, model, tokenizer):
  """
  texts: list of original texts
  """
  device = model.device
  tokens = tokenizer(texts, padding=True, truncation=True, max_length=32, return_tensors="pt")
  input_ids = tokens["input_ids"].to(device)
  attention_mask = tokens["attention_mask"].to(device)
  with torch.no_grad():
      outputs = model(input_ids, attention_mask=attention_mask)
      embeddings = outputs[0]
  return embeddings


In [26]:
def cosine_sim(true_embeddings, pred_embeddings):
  cosine_similarities = []
  for i in range(len(true_embeddings)):
    true_emb = true_embeddings[i].unsqueeze(0).detach().cpu().numpy()
    pred_emb = pred_embeddings[i].unsqueeze(0).detach().cpu().numpy()
    cos_sim = cosine_similarity(true_emb, pred_emb)[0][0]
    cosine_similarities.append(cos_sim)

  avg_cossim = np.mean(cosine_similarities)
  return cosine_similarities, avg_cossim

Now let's calculate cosine similarities between our true and predicted sentences

## Calculating baseline metrics

Baseline: just deleting toxic words from a sentence

In [6]:
toxic_words = pd.read_csv('/content/drive/MyDrive/Машинка/toxic_words.csv')
toxic_words = list(toxic_words['word'])

In [7]:
test_comments = pd.read_csv('/content/drive/MyDrive/Машинка/test_comments.csv')
test_comments = test_comments.dropna()
test_comments.head()

,Unnamed: 0,index,comment,toxic
0,4865,27328,чмо воспитаный ходить жуйка жу т навешивать ше...,1
1,27216,149944,правда написать сделать вывод объективно подоб...,1
2,10110,56610,слово пиздануть,1
3,3924,21896,пиздабол поискать,1
4,26912,149268,затыкаться мочь,1


In [8]:
test_texts = list(test_comments['comment'])

In [9]:
def delete_toxic_words(sentence, toxic_words):
  """
  sentence: string

  toxic_words: list

  """
  sent_list = sentence.split()
  result = []
  for token in sent_list:
    if token in toxic_words:
      continue
    result.append(token)

  return ' '.join(result)


In [10]:
def baseline_predictions(texts, toxic_words):
  """
  texts: list

  toxic_words: list
  """
  clean_texts = []
  for text in texts:
    clean_text = delete_toxic_words(text, toxic_words)
    clean_texts.append(clean_text)
  return clean_texts


In [11]:
baseline_test = baseline_predictions(test_texts, toxic_words)

Calculating STA

In [12]:
# getting model and tokenizer
tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/Машинка/tokenizer")
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Машинка/model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [13]:
accuracy_baseline, baseline_embeddings = STA(tokenizer, model, baseline_test, np.ones(len(baseline_test), dtype = np.int64))

In [15]:
accuracy_baseline

0.40820410205102553

Calculate cosine similarity

In [23]:
pred_embeddings = embeddings(test_texts, model, tokenizer)

In [24]:
true_embeddings = pred_embeddings

In [30]:
# save true embeddings
# import pickle
# with open('/content/drive/MyDrive/Машинка/true_embeddings.pkl', 'wb') as f:
  #  pickle.dump(true_embeddings, f)

In [ ]:
# load true embeddings - this is done because we reload Colab
with open('/content/drive/MyDrive/Машинка/true_embeddings.pkl', 'rb') as f:
    true_embeddings = pickle.load(f)


In [27]:
cosine_similarities, avg_cossim = cosine_sim(true_embeddings, baseline_embeddings)

In [28]:
print(avg_cossim)

-0.13730684


Calculate BLEU

In [35]:
bleu_scores_baseline, avg_blue_baseline = bleu_score(test_texts, baseline_test)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [36]:
print(avg_blue_baseline)

0.2556643356785611
